# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, DatasetDict

/itf-fi-ml/home/krimhau/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.1.0+cu121
CUDA version: 12.1
cuDNN version: 8902
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.35.0
Datasets version: 2.14.6


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Mon Mar 11 10:32:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:41:00.0 Off |                  N/A |
| 30%   48C    P2             117W / 350W |  16552MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
|  1 |  0% |  0% |
|  2 |  0% | 10% |
|  3 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
|  1 |  0% |  0% |
|  2 |  5% | 12% |
|  3 |  0% |  0% |


In [8]:
# Smaller and faster than bert.
model_ckpt = "microsoft/mdeberta-v3-base"

epochs = 5 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [9]:
dataset = load_dataset("kristmh/highest_vs_rest")
dataset

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 937
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 7487
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 935
    })
})

In [10]:
rust = load_dataset("kristmh/rust_testset_with_med_low_1")
rust

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 1572
    })
})

In [11]:
flutter = load_dataset("kristmh/flutter_testset_with_med_low_1")
flutter

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 2370
    })
})

In [12]:
mypy = load_dataset("kristmh/mypy_testset_with_med_low_1")
mypy

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 734
    })
})

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

## Tokenization

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/itf-fi-ml/home/krimhau/miniconda3/envs/thesis/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


    Tokenizing the whole dataset

In [17]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [18]:
rust_tokenized = rust.map(tokenize, batched=True)
rust_testset = rust_tokenized["test"]
rust_testset

Map:   0%|                                                     | 0/1572 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|████████████████████████████████████████| 1572/1572 [00:01<00:00, 1439.39 examples/s]


Dataset({
    features: ['text_clean', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1572
})

In [19]:
flutter_tokenized = flutter.map(tokenize, batched=True)
flutter_testset = flutter_tokenized["test"]
flutter_testset

Map: 100%|████████████████████████████████████████| 2370/2370 [00:02<00:00, 1041.56 examples/s]


Dataset({
    features: ['text_clean', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2370
})

In [20]:
mypy_tokenized = mypy.map(tokenize, batched=True)
mypy_testset = mypy_tokenized["test"]
mypy_testset

Map: 100%|██████████████████████████████████████████| 734/734 [00:00<00:00, 1375.95 examples/s]


Dataset({
    features: ['text_clean', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 734
})

In [21]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map: 100%|██████████████████████████████████████████| 935/935 [00:00<00:00, 2384.27 examples/s]


DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 937
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7487
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 935
    })
})

In [22]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean", "__index_level_0__"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7487
})
Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 935
})


Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 937
})

## Training a classifier

In [23]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
trainer.train() 

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 196.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 25.75 MiB is free. Process 432128 has 2.51 GiB memory in use. Including non-PyTorch memory, this process has 21.14 GiB memory in use. Of the allocated memory 18.55 GiB is allocated by PyTorch, and 2.29 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [ ]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [ ]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

In [ ]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

In [ ]:
rust_test_results = trainer.evaluate(eval_dataset=rust_testset)
for key, value in sorted(rust_test_results.items()):
    print(f"{key} = {value}\n")    

In [ ]:
flutter_test_results = trainer.evaluate(eval_dataset=flutter_testset)
for key, value in sorted(flutter_test_results.items()):
    print(f"{key} = {value}\n")

In [ ]:
mypy_test_results = trainer.evaluate(eval_dataset=mypy_testset)
for key, value in sorted(mypy_test_results.items()):
    print(f"{key} = {value}\n")

In [ ]:
trainer.save_model(model_dir + "_local") 

In [ ]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [ ]:
classifier.model

In [ ]:
classifier("this does not need to be done fast")

In [ ]:
classifier("this is super important")

In [ ]:
classifier("this bug has super high impact on the project")

## Important to delete large objects to free memory 
del train_dataset

In [ ]:
del validation_dataset 

In [ ]:
del model

In [ ]:
# Free cache
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi